
![20230731150911](https://cdn.jsdelivr.net/gh/Corner430/Picture1/images/20230731150911.png)


![20230731151028](https://cdn.jsdelivr.net/gh/Corner430/Picture1/images/20230731151028.png)


![20230731152937](https://cdn.jsdelivr.net/gh/Corner430/Picture1/images/20230731152937.png)

-------------------------------------

### 计算图
- 将代码分解成操作子
- 将计算表示成一个无环图
- 显示构造
  - Tensorflow/Theano/MXNet
- 隐式构造
  - PyTorch/MXNet

![20230731153656](https://cdn.jsdelivr.net/gh/Corner430/Picture1/images/20230731153656.png)

### 自动求导的两种模式

![20230731154100](https://cdn.jsdelivr.net/gh/Corner430/Picture1/images/20230731154100.png)

### 反向累积

![20230731154459](https://cdn.jsdelivr.net/gh/Corner430/Picture1/images/20230731154459.png)

### 反向累积总结
- 构造计算图
- 前向：执行图，存储中间结果
- 反向：从相反方向执行图
  - 去除不需要的枝

![20230731154759](https://cdn.jsdelivr.net/gh/Corner430/Picture1/images/20230731154759.png)

### 复杂度
- 计算复杂度：O(n)，n是操作子个数
  - 通常正向和反向的代价类似
- 内存复杂度：O(n)，因为需要存储正向的所有中间结果（**这正是耗GPU的祸根所在**）
- 跟正向累积相比：
  - O(n)计算复杂度用来计算一个变量的梯度
  - O(1)内存复杂度

> 也就是说正向累积不需要存储中间结果，但是计算复杂度高，而自动求导需要存储中间结果，但是计算复杂度低

----------------------------------------------------------

### 自动求导
假设我们想对函数 $y=2\mathbf{x}^{\top}\mathbf{x}$ 关于列向量 $\mathbf{x}$求导

In [1]:
import torch

x = torch.arange(4.0)
x

tensor([0., 1., 2., 3.])

In [3]:
# 在我们计算y关于x的梯度之前，我们需要一个地方来存储梯度。
x.requires_grad_(True)  # 等价于 x = torch.arange(4.0, requires_grad=True)
x.grad  # 默认为None

In [4]:
# 现在让我们计算y
y = 2 * torch.dot(x, x)
y

tensor(28., grad_fn=<MulBackward0>)

In [5]:
# 通过调用反向传播函数来自动计算y关于x每个分量的梯度
y.backward()
x.grad

tensor([ 0.,  4.,  8., 12.])

In [7]:
x.grad == 4 * x

tensor([True, True, True, True])

In [8]:
# 现在让我们计算x的另一个函数
# 在默认情况下，PyTorch会累积梯度，我们需要清除之前的梯度
x.grad.zero_()
y = x.sum()
y.backward()
x.grad

tensor([1., 1., 1., 1.])

### **深度学习中，我们的目的不是计算微分矩阵，而是批量中每个样本单独计算的偏导数之和**

In [9]:
# 对非标量调用`backward`需要传入一个`gradient`参数，该参数指定微分函数关于`self`的梯度。
x.grad.zero_()
y = x * x
# 等价于y.backward(torch.ones(len(x)))
y.sum().backward()
x.grad

tensor([0., 2., 4., 6.])

In [11]:
# 将某些计算移动到记录的计算图之外
x.grad.zero_()
y = x * x
u = y.detach()  # 将y当作一个常数，不参与计算图的构建
z = u * x

z.sum().backward()
x.grad == u

tensor([True, True, True, True])

In [12]:
x.grad.zero_()
y.sum().backward()
x.grad == 2 * x

tensor([True, True, True, True])

### **即使构建函数的计算图需要通过Python控制流（例如，条件、循环或任意函数调用），我们仍然可以计算得到的变量的梯度**

In [16]:
def f(a):
    b = a * 2
    while b.norm() < 1000:
        b = b * 2
    if b.sum() > 0:
        c = b
    else:
        c = 100 * b
    return c

a = torch.randn(size=(), requires_grad=True)    # size=()表示标量
d = f(a)
d.backward()

a.grad == d / a

tensor(True)